# Radial Distribution Function


The radial distribution function in 2D is:
$$ g_2(r) = \frac{n(r)}{\pi [(r+\delta r)^2 - r^2 ] \rho} $$

where the density is $\rho = N/V $ and the number density $n(r)$ is computed (using a KDTree data structure) by taking the number of elements closer than $r + \delta r$ minus the number of elements closer than $r$

In 3D (i think):
$$ S_r = 4 \pi r^2 g_2(r) $$

in 2D ???

## regular

In [ ]:
@joblib.delayed
def rdf_frame(frame, COORDS, rList, dr, rho):
    coords = COORDS[frame*nDrops:(frame+1)*nDrops,:]
    kd = KDTree(coords)

    avg_n = np.zeros(len(rList))
    for i, r in enumerate(rList):
        a = kd.query_ball_point(coords, r + 20)
        b = kd.query_ball_point(coords, r)
        
        n1 = 0
        for j in a:
            n1 += len(j) - 1

        n2 = 0
        for j in b:
            n2 += len(j) - 1
        
        avg_n[i] = n1/len(a) - n2/len(b)

    rdf = avg_n/(np.pi*(dr**2 + 2*rList*dr)*rho)
    return rdf

In [ ]:
def get_rdf(run_analysis_verb, nFrames, trajs, rList, dr, rho):
    if run_analysis_verb:
        COORDS = np.array(trajs.loc[:, ["x","y"]])
        parallel = joblib.Parallel(n_jobs = -2)
        frames = nFrames
        rdf = parallel(
            rdf_frame(frame, COORDS, rList, dr, rho)
            for frame in tqdm( range(frames) )
        )
        rdf = np.array(rdf)
    else:
        rdf = np.array(pd.read_csv("./analysis_data/rdf.csv"))
    return rdf

In [ ]:
dr = 5
rDisk = 822/2
rList = np.arange(0, 2*rDisk, 1)
rho = nDrops/(np.pi*rDisk**2) # nDrops - 1 !

print("Radial Distribution Function ")
rdf = get_rdf(run_analysis_verb, nFrames, rawTrajs, rList, dr, rho)

In [ ]:
if 0:
    fig, ax = plt.subplots()
    anim_running = True

    def onClick(event):
        global anim_running
        if anim_running:
            ani.event_source.stop()
            anim_running = False
        else:
            ani.event_source.start()
            anim_running = True

    line, = ax.plot(rList, rdf[0])
    title = ax.set_title('Test, time=0')

    def animate(frame):
        line.set_ydata(rdf[frame])  # update the data.
        title.set_text('Test, time={}'.format(frame))
        return line, 

    fig.canvas.mpl_connect('button_press_event', onClick)
    ani = matplotlib.animation.FuncAnimation(fig, animate, range(0, rdf.shape[0]), interval=20, blit=False)
    if save_verb: ani.save('./results/radial_distribution_function/rdf.mp4', fps=60, extra_args=['-vcodec', 'libx264'])
    if show_verb:
        plt.show()
    else:
        plt.close()

In [ ]:
g_plot = rdf[::30, :].T

timearr = np.linspace(0, rdf.shape[0], 10)/10
timearr = timearr.astype(int)

fig, ax = plt.subplots(1, 1, figsize=(8,6))
img = ax.imshow(np.log(1 + g_plot))
ax.set_xticks(np.linspace(0, g_plot.shape[1], 10))
ax.set_yticks(np.linspace(0, g_plot.shape[0], 10))
ax.set_xticklabels(timearr)
ax.set_yticklabels(np.linspace(0, 2*rDisk, 10).astype(int))
fig.colorbar(img)
plt.xlabel("Time [s]")
plt.ylabel("r [px]")
plt.title("$Log(1 + g_2)$ heatmap ")

if save_verb: plt.savefig("./results/radial_distribution_function/rdf_heatmap.png", bbox_inches='tight')
if show_verb: 
    plt.show()
else:
    plt.close()

## RDF FROM CENTER

In [ ]:
@joblib.delayed
def rdf_center_frame(frame, COORDS, r_c, rList, dr, rho):
    coords = COORDS[frame*nDrops:(frame+1)*nDrops,:]
    kd = KDTree(coords)
    avg_n = np.zeros(len(rList))
    for i, r in enumerate(rList):
        # find all the points within r+dr
        a = kd.query_ball_point(r_c, r + dr)
        n1 = len(a) 
        # find all the points within r+dr
        b = kd.query_ball_point(r_c, r)
        n2 = len(b)
        avg_n[i] = n1 - n2
    rdf = avg_n/(np.pi*(dr**2 + 2*rList*dr)*rho)
    return rdf

def get_rdf_center(run_analysis_verb, nFrames, trajs, r_c, rList, dr, rho):
    if run_analysis_verb:
        COORDS = np.array(trajs.loc[:,["x","y"]])
        parallel = joblib.Parallel(n_jobs = -2)
        rdf_c = parallel(
            rdf_center_frame(frame, COORDS, r_c, rList, dr, rho)
            for frame in tqdm( range(nFrames) )
        )
        rdf_c = pd.DataFrame(np.array(rdf_c))
        rdf_c.columns = [str(i) for i in rList]
        pd.DataFrame(rdf_c).to_parquet("./analysis_data/rdf_centre.parquet")
    else:
        rdf_c = pd.read_parquet("./analysis_data/rdf_centre.parquet")
    return np.array(rdf_c)

In [ ]:
dr = 20
rDisk = 822/2
rList = np.arange(0, rDisk, 1)
rho = nDrops/(np.pi*rDisk**2) # nDrops -1 !
r_c = [470, 490] #centre of the image --> to refine

print("Radial Distribution Function from the center")
rdf_c = get_rdf_center(run_analysis_verb, nFrames, rawTrajs, r_c, rList, dr, rho)

In [ ]:
fig, ax = plt.subplots()
ax.plot(rList, rdf_c.mean(axis=0), label="mean")
ax.fill_between(rList, rdf_c.mean(axis=0) - rdf_c.std(axis=0), rdf_c.mean(axis=0) + rdf_c.std(axis=0), alpha=0.3, label="std")
ax.set_xlabel("r [px]")
ax.set_ylabel("g(r)")
ax.legend()
plt.title("Radial Distribution Function from the center")
if save_verb: plt.savefig("./results/radial_distribution_function/rdf_center.png", bbox_inches='tight')
if show_verb: 
    plt.show()
else:
    plt.close()

In [ ]:
if animated_plot_verb:
    fig, ax = plt.subplots()
    anim_running = True

    def onClick(event):
        global anim_running
        if anim_running:
            ani.event_source.stop()
            anim_running = False
        else:
            ani.event_source.start()
            anim_running = True

    line, = ax.plot(rList, rdf_c[0])
    title = ax.set_title('Test, time=0')
    ax.set_ylim(-1, 10)

    def animate(frame):
        line.set_ydata(rdf_c[frame])  # update the data.
        title.set_text('Test, time={}'.format(frame))
        return line, 

    fig.canvas.mpl_connect('button_press_event', onClick)
    ani = matplotlib.animation.FuncAnimation(fig, animate, range(0, frames), interval=5, blit=False)
    if save_verb: ani.save('./results/radial_distribution_function/rdf_from_centre.mp4', fps=60, extra_args=['-vcodec', 'libx264'])
    if show_verb:
        plt.show()
    else:
        plt.close()

In [ ]:
g_plot = rdf_c[::30, :].T

timearr = np.linspace(0, nFrames, 5)/10
timearr = timearr.astype(int)

fig, ax = plt.subplots(1, 1, figsize=(8,6))
img = ax.imshow(np.log(1+g_plot))
ax.set_xticks(np.linspace(0, g_plot.shape[1], 5))
ax.set_yticks(np.linspace(0, g_plot.shape[0], 5))
ax.set_xticklabels(timearr)
ax.set_yticklabels(np.linspace(0, rDisk, 5).astype(int))
ax.set_aspect(2)
fig.colorbar(img)
plt.xlabel("Time [s]")
plt.ylabel("r [px]")
plt.title("$Log(1 + g_2)$ heatmap ")
if save_verb: plt.savefig("./results/radial_distribution_function/rdf_heatmap_from_center.png", bbox_inches='tight')
if show_verb:
    plt.show()
else:
    plt.close()